In [1]:
import json
import os
import requests

import pandas as pd

from CONSTANTS import BASE_URL, PROJECT_DIR

In [2]:
OUT_DIR = os.path.join(PROJECT_DIR, "output")
if not os.path.isdir(OUT_DIR):
    os.makedirs(OUT_DIR)

In [3]:
def get_from_dt():
    return "01/01/2021"

def get_to_dt():
    return "27/11/2021"

def get_script():
    return "500325"

In [4]:
url = f"{BASE_URL}/InsiderTrade15/w"
from_dt = get_from_dt()
to_dt = get_to_dt()
script_id = get_script()
params = {
    "fromdt": from_dt,
    "todt": to_dt,
    "scripcode": script_id
}
url2 = f"{url}?todt={to_dt}&scripcode={script_id}&fromdt={from_dt}"
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:94.0) Gecko/20100101 Firefox/94.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "DNT": "1",
    "Connection": "keep-alive",
    "Cookie": "_ga_TM52BJH9HF=GS1.1.1638520636.1.0.1638520636.0; _ga=GA1.1.1995187650.1638520636",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Sec-GPC": "1",
    "Cache-Control": "max-age=0",
    "TE": "trailers"
}
payload = {}

In [5]:
resp = requests.get(url, params=params, headers=headers, data=payload)
resp = resp.json()

In [6]:
def make_filings(raw_filings):
    filings = []
    promoters = {}
    for filing in raw_filings:
        pan = filing["Fld_PromoterPAN"]
        if pan not in promoters:
            promoters[pan] = {
                "pan": pan,
                "name": filing["Fld_PromoterName"],
                "address": filing["Fld_PromoterAdd"],
                "contact_no": filing["Fld_PromoterContact"],
                "CIN": filing["Fld_PromoterCIN"],
                "DIN": filing["Fld_PromoterDIN"],
                "category": filing["Fld_PromoterCatg"],
                "category_name": filing["Fld_PersonCatgName"]
            }
        filings.append({
            "script_id": str(filing["Fld_ScripCode"]),
            "company_name": filing["Companyname"],
            "promoter_pan": pan,
            "security": {
                "number": filing["Fld_SecurityNo"],
                "value": filing["Fld_SecurityValue"],
                "type": filing["Fld_SecurityType"],
                "type_name": filing["Fld_SecurityTypeName"],
                "prior": {
                    "number": filing["Fld_SecurityNoPrior"],
                    "type": filing["Fld_SecurityTypePrior"],
                    "type_name": filing["Fld_SecurityTypeNamePrior"]
                },
                "post": {
                    "number": filing["Fld_SecurityNoPost"],
                    "type": filing["Fld_SecurityTypePost"]
                }
            },
            "transaction": {
                "mode": filing["Fld_Mode"],
                "mode_name": filing["ModeOfAquisation"],
                "buy_value": filing["Fld_TradeDerivBuyValue"],
                "buy_unints": filing["Fld_TradeDerivBuyUnits"],
                "sell_value": filing["Fld_TradeDerivSellValue"],
                "sell_units": filing["Fld_TradeDerivSellUnits"],
                "exchange": filing["Fld_TradeExchange"],
                "from_date": filing["Fld_FromDate"],
                "to_date": filing["Fld_ToDate"],
                "allotment_from_date": filing["Fld_FromDateAllotment"],
                "allotment_to_date": filing["Fld_ToDateAllotment"],
                "intimation_date": filing["Fld_DateIntimation"],
                "create_date": filing["Fld_CreateDate"],
                "update_date": filing["Fld_UpdateDate"],
                "authorise_date": filing["Fld_AuthoriseDate"],
                "letter_date": filing["Fld_LetterDate"],
                "stamp_date": filing["Fld_StampDate"]
            },
            "contract_type": filing["Fld_TypeofContract"],
            "attachment": filing["Fld_attachment"],
            "attachment_ICRA": filing["Fld_AttachmentICRA"]
        })
    return filings, promoters

def save_filings(filings):
    with open(os.path.join(OUT_DIR, "filings.json"), "w") as f:
        json.dump(filings, f)

def save_promoters(promoters):
    with open(os.path.join(OUT_DIR, "promoters.json"), "w") as f:
        json.dump(promoters, f)

In [7]:
filings, promoters = make_filings(resp["Table"])
save_filings(filings)
save_promoters(promoters)

In [8]:
def get_filings():
    with open(os.path.join(OUT_DIR, "filings.json")) as f:
        return json.load(f)

def get_promoters():
    with open(os.path.join(OUT_DIR, "promoters.json")) as f:
        return json.load(f)

def generate_insider_trading_report():
    filings = get_filings()
    promoters = get_promoters()
    data = []
    cols = [
        "script_id", "company_name", "promoter_pan", "promoter_name", "promoter_address",
        "promoter_contact_no", "promoter_CIN", "promoter_DIN", "promoter_category",
        "promoter_category_name", "security_number", "security_value", "security_type",
        "security_type_name", "security_prior_number", "security_prior_type",
        "security_prior_type_name", "security_post_number", "security_post_type",
        "transaction_mode", "transaction_mode_name", "transaction_buy_value",
        "transaction_buy_unints", "transaction_sell_value", "transaction_sell_units",
        "transaction_exchange", "transaction_from_date", "transaction_to_date",
        "transaction_allotment_from_date", "transaction_allotment_to_date",
        "transaction_intimation_date", "transaction_create_date", "transaction_update_date",
        "transaction_authorise_date", "transaction_letter_date", "transaction_stamp_date",
        "contract_type", "attachment", "attachment_ICRA"
    ]
    for filing in filings:
        pan = filing["promoter_pan"]
        data.append([
            filing["script_id"], filing["company_name"], promoters[pan]["pan"],
            promoters[pan]["name"], promoters[pan]["address"], promoters[pan]["contact_no"],
            promoters[pan]["CIN"], promoters[pan]["DIN"], promoters[pan]["category"],
            promoters[pan]["category_name"], filing["security"]["number"],
            filing["security"]["value"], filing["security"]["type"],
            filing["security"]["type_name"], filing["security"]["prior"]["number"],
            filing["security"]["prior"]["type"], filing["security"]["prior"]["type_name"],
            filing["security"]["post"]["number"], filing["security"]["post"]["type"],
            filing["transaction"]["mode"], filing["transaction"]["mode_name"],
            filing["transaction"]["buy_value"], filing["transaction"]["buy_unints"],
            filing["transaction"]["sell_value"], filing["transaction"]["sell_units"],
            filing["transaction"]["exchange"], filing["transaction"]["from_date"],
            filing["transaction"]["to_date"], filing["transaction"]["allotment_from_date"],
            filing["transaction"]["allotment_to_date"], filing["transaction"]["intimation_date"],
            filing["transaction"]["create_date"], filing["transaction"]["update_date"],
            filing["transaction"]["authorise_date"], filing["transaction"]["letter_date"],
            filing["transaction"]["stamp_date"], filing["contract_type"], filing["attachment"],
            filing["attachment_ICRA"]
        ])
    return pd.DataFrame(data, columns=cols)

In [9]:
df = generate_insider_trading_report()
df.head()

,script_id,company_name,promoter_pan,promoter_name,promoter_address,promoter_contact_no,promoter_CIN,promoter_DIN,promoter_category,promoter_category_name,...,transaction_allotment_to_date,transaction_intimation_date,transaction_create_date,transaction_update_date,transaction_authorise_date,transaction_letter_date,transaction_stamp_date,contract_type,attachment,attachment_ICRA
0,500325,RELIANCE INDUSTRIES LTD.,AAKPN1989R,DARSHAN PURUSHOTTAM NAPHADE,"602 MANGAL PRABHA, SECTOR 9, D. Y. PATIL HOSPI...",9867625794,,,6,Designated Person,...,2021-09-29T00:00:00,2021-10-01T00:00:00,2021-10-04T19:58:43.803,2021-10-04T20:01:57,2021-10-04T20:02:00,2021-10-04T00:00:00,2021-10-04T00:00:00,,None,
1,500325,RELIANCE INDUSTRIES LTD.,ACNPB1702F,Chandra Shekhar Borar,"C 1003, CHAITANYA TOWER, 10TH FLOOR,APPA SAHEB...",9867550242,,,6,Designated Person,...,2021-09-24T00:00:00,2021-09-28T00:00:00,2021-09-30T23:29:14.463,2021-10-01T12:48:22,2021-10-01T12:48:24,2021-09-30T00:00:00,2021-09-30T00:00:00,,None,
2,500325,RELIANCE INDUSTRIES LTD.,ACNPB1702F,Chandra Shekhar Borar,"C 1003, CHAITANYA TOWER, 10TH FLOOR,APPA SAHEB...",9867550242,,,6,Designated Person,...,2021-09-23T00:00:00,2021-09-26T00:00:00,2021-09-28T20:05:35.977,2021-09-28T20:14:03,2021-09-28T20:14:04,2021-09-28T00:00:00,2021-09-28T00:00:00,,None,
3,500325,RELIANCE INDUSTRIES LTD.,ACNPB1702F,Chandra Shekhar Borar,"C 1003, CHAITANYA TOWER, 10TH FLOOR,APPA SAHEB...",9867550242,,,6,Designated Person,...,2021-09-07T00:00:00,2021-09-08T00:00:00,2021-09-13T19:41:54.24,2021-09-13T21:37:16,2021-09-13T21:37:17,2021-09-13T00:00:00,2021-09-13T00:00:00,,None,
4,500325,RELIANCE INDUSTRIES LTD.,ACNPB1702F,Chandra Shekhar Borar,"C 1003, CHAITANYA TOWER, 10TH FLOOR,APPA SAHEB...",9867550242,,,6,Designated Person,...,2021-09-07T00:00:00,2021-09-08T00:00:00,2021-09-13T19:41:54.24,2021-09-13T21:37:16,2021-09-13T21:37:17,2021-09-13T00:00:00,2021-09-13T00:00:00,,None,


In [10]:
df.to_csv(os.path.join(OUT_DIR, "insider_trading.csv"), index=False)